# Change FEniCS Form Compiler Scheme

Changing the form compiler scheme is sometimes necessary when we use some special quadrature.

Here I introduce two ways to change the compiler scheme. 

The example comes from the attempt to get lumped matrix using GLL quadrature.

In [1]:
from dolfin import *
import numpy as np
np.set_printoptions(precision=3)

In [2]:
mesh = UnitIntervalMesh(3)
V = FunctionSpace(mesh, 'CG', 2)  # 2nd order CG == 2nd order GLL
M = assemble(TrialFunction(V)*TestFunction(V)*dx)
M.array()  # The mass matrix under standard quadrature scheme is not diagonal.

array([[ 0.044, -0.011,  0.022,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.011,  0.089,  0.022, -0.011,  0.022,  0.   ,  0.   ],
       [ 0.022,  0.022,  0.178,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.011,  0.   ,  0.089,  0.022, -0.011,  0.022],
       [ 0.   ,  0.022,  0.   ,  0.022,  0.178,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , -0.011,  0.   ,  0.044,  0.022],
       [ 0.   ,  0.   ,  0.   ,  0.022,  0.   ,  0.022,  0.178]])

If we want to use GLL quadrature, first we need to hack the fenics package to add a scheme.

In [3]:
from numpy import array
from FIAT.reference_element import UFCInterval, UFCQuadrilateral
from FIAT.quadrature import QuadratureRule, GaussLobattoLegendreQuadratureLineRule, GaussJacobiQuadratureLineRule
from FIAT.quadrature_schemes import create_quadrature
from ffc.analysis import _autoselect_quadrature_rule

def create_quadrature_monkey_patched(ref_el, degree, scheme="default"):
    """Monkey patched FIAT.quadrature_schemes.create_quadrature()
    for implementing lumped scheme"""
    # Our "special" scheme
    if scheme == "GLL":
        # print('Use GLL quadrature rule')
        if isinstance(ref_el, UFCInterval):
            # print('Integrate on intervals')
            return GaussLobattoLegendreQuadratureLineRule(ref_el, degree + 1)
        elif isinstance(ref_el, UFCQuadrilateral):
            # print('Integrate on quadrilaterals')
            lr = GaussLobattoLegendreQuadratureLineRule(UFCInterval(), degree + 1)
            x = lr.pts
            w = lr.wts
            x = array([[i[0], j[0]] for i in x for j in x])
            w = array([i * j for i in w for j in w])
            return QuadratureRule(ref_el, x, w)


    # Fallback to FIAT's normal operation
    return create_quadrature(ref_el, degree, scheme=scheme)


def _autoselect_quadrature_rule_monkey_patched(*args, **kwargs):
    """Monkey patched ffc.analysis._autoselect_quadrature_rule()
    preventing FFC to complain about non-existing quad scheme"""
    try:
        return _autoselect_quadrature_rule(*args, **kwargs)
    except Exception:
        integral_metadata = args[0]
        qr = integral_metadata["quadrature_rule"]
        return qr


# Monkey patch FIAT quadrature scheme generator
import FIAT

FIAT.create_quadrature = create_quadrature_monkey_patched

# Monkey patch FFC scheme autoselection mechanism
import ffc.analysis

ffc.analysis._autoselect_quadrature_rule = _autoselect_quadrature_rule_monkey_patched

# # cheat ffc to support GLL
# from ffc import fiatinterface

# supported_families_monkey_patched = list(fiatinterface.supported_families)
# supported_families_monkey_patched.append('Gauss-Lobatto-Legendre')
# fiatinterface.supported_families = tuple(supported_families_monkey_patched)

Now we can assemble the mass matrix with GLL scheme and get a diagonal matrix.
## Method 1: set the arguments in the function `assemble`

In [4]:
M = assemble(TrialFunction(V)*TestFunction(V)*dx, 
             form_compiler_parameters={'quadrature_degree': 2, 'quadrature_rule': 'GLL'})
M.array()  # The mass matrix under GLL quadrature scheme is diagonal.

array([[ 0.056,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.111,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.222,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.111,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.222,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.056,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.222]])

However it is bothering to set arguments every time when we keep using this scheme in the whole project. Hence here I introduce another way, changing the dolfin global parameter which controls the compile scheme.
## Method 2: Set global `parameters`

In [5]:
info(parameters,True)  # It prints the global dolfin parameters. It doesn't work here, why?

In [6]:
# change the parameters 
parameters['form_compiler']['quadrature_degree'] = 2
parameters['form_compiler']['quadrature_rule'] = 'GLL'

In [7]:
M = assemble(TrialFunction(V)*TestFunction(V)*dx)
M.array()  # The mass matrix under standard quadrature scheme is not diagonal.

array([[ 0.056,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.111,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.222,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.111,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.222,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.056,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.222]])